In [1]:
import json
import requests
import urllib.request
import io

from IPython.display import display, Math, Latex, Markdown, clear_output, HTML

import matplotlib.pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

from tqdm.notebook import tqdm

from scipy.optimize import curve_fit
from scipy import stats
import numpy as np
import pandas as pd

In [2]:
_R = pd.read_pickle("double-gompertz-data.pkl")

In [3]:
errors = []
for country in _R:
    print(f"----> CHECKING {country}")
    for function in _R[country].index:
        if _R[country][function] is None:
            print(f"! ERROR {country} with {function}")
            if country not in errors:
                errors.append(country)

----> CHECKING Afghanistan
----> CHECKING Albania
----> CHECKING Algeria
----> CHECKING Andorra
----> CHECKING Angola
----> CHECKING Antigua and Barbuda
----> CHECKING Argentina
----> CHECKING Armenia
----> CHECKING Australia
----> CHECKING Austria
----> CHECKING Azerbaijan
----> CHECKING Bahamas
----> CHECKING Bahrain
----> CHECKING Bangladesh
! ERROR Bangladesh with dobgomp
----> CHECKING Barbados
----> CHECKING Belarus
----> CHECKING Belgium
----> CHECKING Belize
----> CHECKING Benin
----> CHECKING Bhutan
! ERROR Bhutan with dobgomp
----> CHECKING Bolivia
----> CHECKING Bosnia and Herzegovina
----> CHECKING Botswana
----> CHECKING Brazil
! ERROR Brazil with doblogit
----> CHECKING Brunei
----> CHECKING Bulgaria
----> CHECKING Burkina Faso
----> CHECKING Burma
! ERROR Burma with dobgomp
----> CHECKING Burundi
----> CHECKING Cabo Verde
! ERROR Cabo Verde with doblogit
----> CHECKING Cambodia
----> CHECKING Cameroon
----> CHECKING Canada
----> CHECKING Central African Republic
----> CH

In [4]:
print(f"COUNTRIES WITH ERRORS: {len(errors)}")
print(f"COUNTRIES TO BE USED: {len(_R.columns) - len(errors)}")

COUNTRIES WITH ERRORS: 17
COUNTRIES TO BE USED: 168


In [93]:
def get_stats(min_cases=100):

    total_results = {
        "Function": [],
        "Nations": [],
        "Days": [],
        "Samples": [],
        "Slope": [],
        "Intercept": [],
        "Rho": [],
        "p-value": [],
        "Std Err": [],
    }
    total_data = {}

    for function in tqdm(_R.index, leave=False):
        if function == "measured":
            continue
        RSD = {"country": [], "slope": [], "intercept": [], "r_value": [], "p_value": [], "std_err": []}
        for country in tqdm(_R.columns, leave=False):
            if country in errors:
                continue
            if _R[country]["measured"]["valore"].tail(1).values[0] < min_cases:
                continue
            slope, intercept, r_value, p_value, std_err = stats.linregress(
                _R[country]["measured"]["valore"],
                _R[country][function]["valore"]
            )
            if slope == 1.0 or intercept == 0.0 or r_value == 1.0 or p_value == 0.0 or std_err == 0.0:
                if country not in errors:
                    errors.append(country)
                    print(f"--------> ERROR {country}")
                    continue
            RSD["country"].append(country)
            RSD["slope"].append(slope)
            RSD["intercept"].append(intercept)
            RSD["r_value"].append(r_value)
            RSD["p_value"].append(p_value)
            RSD["std_err"].append(std_err)

        RSDdf = pd.DataFrame(data=RSD)
        mean_slope = np.mean(RSDdf['slope'])
        mean_intercept = np.mean(RSDdf['intercept'])
        mean_r_value = np.mean(RSDdf['r_value'])
        mean_p_value = np.mean(RSDdf['p_value'])
        mean_std_err = np.mean(RSDdf['std_err'])

        total_results["Function"].append(function)
        total_results["Nations"].append(RSDdf["country"].size)
        total_results["Days"].append(_R[country]["measured"]["valore"].size)
        total_results["Samples"].append(RSDdf["country"].size * _R[country]["measured"]["valore"].size)
        total_results["Slope"].append(mean_slope)
        total_results["Intercept"].append(mean_intercept)
        total_results["Rho"].append(mean_r_value)
        total_results["p-value"].append(mean_p_value)
        total_results["Std Err"].append(mean_std_err)

        total_data.update({function: [RSDdf]})

    TOTdf = pd.DataFrame(data=total_results)
    DATdf = pd.DataFrame(total_data)

    return TOTdf, DATdf

In [94]:
for min_cases in [0, 100, 500, 1000, 2000, 5000, 10000, 20000]:
    display(Markdown(f"## Casi minimi {min_cases}"))
    TOT, DAT = get_stats(min_cases)
    display(HTML(TOT.to_html(notebook=True, col_space=100, columns=[
        "Function", "Nations", "Days", "Samples", "Slope", "Intercept", "Rho", "p-value", "Std Err",
    ], index=False)))
    TOT.to_pickle(f"double-gompertz-stats-{min_cases}-mean.pkl")
    DAT.to_pickle(f"double-gompertz-stats-{min_cases}-data.pkl")

## Casi minimi 0

--------> ERROR Sao Tome and Principe
--------> ERROR Yemen


--------> ERROR MS Zaandam
--------> ERROR South Sudan
--------> ERROR Western Sahara


Function,Nations,Days,Samples,Slope,Intercept,Rho,p-value,Std Err
logit,166,93,15438,0.989002,25.132105,0.995207,4.108973e-46,0.008257
gomp,166,93,15438,0.992591,5.091736,0.995799,4.810899e-47,0.007503
doblogit,163,93,15159,0.996693,2.737228,0.998207,5.268991e-67,0.004997
dobgomp,163,93,15159,0.994428,4.727219,0.996844,1.762817e-45,0.006281


## Casi minimi 100

...

Function,Nations,Days,Samples,Slope,Intercept,Rho,p-value,Std Err
logit,41,93,3813,0.987743,99.983467,0.997559,9.504434e-49,0.005199
gomp,41,93,3813,0.998371,19.291896,0.999185,4.524365e-102,0.003625
doblogit,41,93,3813,0.999239,10.252889,0.999620,6.520655e-104,0.002397
dobgomp,41,93,3813,0.998647,17.691137,0.999296,4.491526e-102,0.003200


## Casi minimi 10000

Function,Nations,Days,Samples,Slope,Intercept,Rho,p-value,Std Err
logit,28,93,2604,0.982971,145.397008,0.996885,1.391721e-48,0.005565
gomp,28,93,2604,0.998272,27.545053,0.999135,6.624963e-102,0.003597
doblogit,28,93,2604,0.999294,14.586274,0.999647,9.548101e-104,0.002215
dobgomp,28,93,2604,0.998623,25.261813,0.999268,6.576877e-102,0.003096


## Casi minimi 20000

Function,Nations,Days,Samples,Slope,Intercept,Rho,p-value,Std Err
logit,16,93,1488,0.972789,246.713997,0.995845,2.435511e-48,0.005453
gomp,16,93,1488,0.998892,43.022343,0.999445,1.241411e-104,0.002849
doblogit,16,93,1488,0.999469,24.092451,0.999735,3.873059e-110,0.001916
dobgomp,16,93,1488,0.999110,37.599265,0.999555,1.241515e-104,0.002360
